## Import Library

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import time
import random
from torch.utils.data import Dataset, DataLoader
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

curPath = os.path.abspath(os.path.dirname('models'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

## Utils

In [2]:
def transfer_labels(labels):
    indicies = np.unique(labels)
    num_samples = labels.shape[0]

    for i in range(num_samples):
        new_label = np.argwhere(labels[i] == indicies)[0][0]
        labels[i] = new_label

    return labels

def collate_fn(data, device, max_len=None):
    """Build mini-batch tensors from a list of (X, mask) tuples. Mask input. Create
    Args:
        data: len(batch_size) list of tuples (X, y).
            - X: torch tensor of shape (seq_length, feat_dim); variable seq_length.
            - y: torch tensor of shape (num_labels,) : class indices or numerical targets
                (for classification or regression, respectively). num_labels > 1 for multi-task models
        max_len: global fixed sequence length. Used for architectures requiring fixed length input,
            where the batch length cannot vary dynamically. Longer sequences are clipped, shorter are padded with 0s
    Returns:
        X: (batch_size, padded_length, feat_dim) torch tensor of masked features (input)
        targets: (batch_size, padded_length, feat_dim) torch tensor of unmasked features (output)
        target_masks: (batch_size, padded_length, feat_dim) boolean torch tensor
            0 indicates masked values to be predicted, 1 indicates unaffected/"active" feature values
        padding_masks: (batch_size, padded_length) boolean tensor, 1 means keep vector at this position, 0 means padding
    """

    batch_size = len(data)
    features, labels = zip(*data)

    # Stack and pad features and masks (convert 2D to 3D tensors, i.e. add batch dimension)
    lengths = [X.shape[0] for X in features]  # original sequence length for each time series
    if max_len is None:
        max_len = max(lengths)

    X = torch.zeros(batch_size, max_len, features[0].shape[-1])  # (batch_size, padded_length, feat_dim)
    for i in range(batch_size):
        end = min(lengths[i], max_len)
        X[i, :end, :] = features[i][:end, :]

    targets = torch.stack(labels, dim=0)  # (batch_size, num_labels)

    padding_masks = padding_mask(torch.tensor(lengths, dtype=torch.int16),
                                 max_len=max_len)  # (batch_size, padded_length) boolean tensor, "1" means keep

    return X.to(device), targets.to(device), padding_masks.to(device)


def padding_mask(lengths, max_len=None):
    """
    Used to mask padded positions: creates a (batch_size, max_len) boolean mask from a tensor of sequence lengths,
    where 1 means keep element at this position (time step)
    """
    batch_size = lengths.numel()
    max_len = max_len or lengths.max_val()  # trick works because of overloading of 'or' operator for non-boolean types
    return (torch.arange(0, max_len, device=lengths.device)
            .type_as(lengths)
            .repeat(batch_size, 1)
            .lt(lengths.unsqueeze(1)))



def evaluate(args, val_loader, model, loss):
    val_loss = 0
    val_accu = 0

    sum_len = 0
    for data, target, padding_x_mask in val_loader:
        '''
        data, target = data.to(device), target.to(device)
        target = target.to(torch.int64)
        '''
        with torch.no_grad():
            val_pred = model(data, padding_x_mask)
            val_loss += loss(val_pred, target).item()
            val_accu += torch.sum(torch.argmax(val_pred.data, axis=1) == target)
            sum_len += len(target)

    return val_loss / sum_len, val_accu / sum_len

def set_seed(args):
    random.seed(args.random_seed)
    np.random.seed(args.random_seed)
    torch.manual_seed(args.random_seed)
    torch.cuda.manual_seed(args.random_seed)
    torch.cuda.manual_seed_all(args.random_seed)


class HARDataset(Dataset):
    def __init__(self, dataset, target):
        # (num_size, num_dimensions, series_length)
        self.dataset = dataset.permute(0, 2, 1)
        self.target = target

    def __getitem__(self, index):
        return self.dataset[index], self.target[index]

    def __len__(self):
        return len(self.target)
    
def normalize(data_set):
    '''
    The function is the same as normalize_per_series, but can be used for multiple variables.
    '''
    return TimeSeriesScalerMeanVariance().fit_transform(data_set)

## Arguments

In [3]:
parser = argparse.ArgumentParser()

# Base setup
parser.add_argument('--backbone', type=str, default='fcn', help='encoder backbone, fcn')
parser.add_argument('--random_seed', type=int, default=42, help='shuffle seed')

# Dataset setup
parser.add_argument('--num_classes', type=int, default=0, help='number of class')
parser.add_argument('--normalize_way', type=str, default='single', help='single or train_set')
# parser.add_argument('--seq_len', type=int, default=46, help='seq_len')
parser.add_argument('--input_size', type=int, default=1, help='input_size')

# parser.add_argument('--patch_size', type=int, default=8, help='patch_size')
# parser.add_argument('--stride', type=int, default=8, help='stride')

parser.add_argument('--target_points', type=int, default=96, help='forecast horizon')

# Patch
parser.add_argument('--patch_len', type=int, default=8, help='patch length')
parser.add_argument('--stride', type=int, default=8, help='stride between patch')

# Semi training
parser.add_argument('--labeled_ratio', type=float, default='0.1', help='0.1, 0.2, 0.4')

# basic config
parser.add_argument('--task_name', type=str, required=False, default='classification',
                    help='task name, options:[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length')
parser.add_argument('--pred_len', type=int, default=0, help='prediction sequence length')
parser.add_argument('--seasonal_patterns', type=str, default='Monthly', help='subset for M4')
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)

# model define
parser.add_argument('--top_k', type=int, default=3, help='for TimesBlock')
parser.add_argument('--num_kernels', type=int, default=6, help='for Inception')
parser.add_argument('--enc_in', type=int, default=7, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=7, help='decoder input size')
parser.add_argument('--c_out', type=int, default=7, help='output size')
parser.add_argument('--d_model', type=int, default=64, help='dimension of model')   ###
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=3, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=64, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=True)
parser.add_argument('--dropout', type=float, default=0.1, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=1, help='gpu')


# training setup
parser.add_argument('--loss', type=str, default='cross_entropy', help='loss function')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0.0, help='weight decay')
parser.add_argument('--batch_size', type=int, default=8, help='')
parser.add_argument('--epoch', type=int, default=15, help='training epoch')
parser.add_argument('--cuda', type=str, default='cuda:1')

parser.add_argument('--save_dir', type=str, default='/SSD/lz/time_series_label_noise/result')
parser.add_argument('--save_csv_name', type=str, default='timesnet_ucr_supervised_0801_')

# classifier setup
parser.add_argument('--classifier', type=str, default='linear', help='type of classifier(linear or nonlinear)')
parser.add_argument('--classifier_input', type=int, default=128, help='input dim of the classifiers')

args = parser.parse_args(args=[])

device = torch.device(args.cuda if torch.cuda.is_available() else "cpu")
set_seed(args)
args

Namespace(backbone='fcn', random_seed=42, num_classes=0, normalize_way='single', input_size=1, target_points=96, patch_len=8, stride=8, labeled_ratio=0.1, task_name='classification', freq='h', seq_len=96, label_len=48, pred_len=0, seasonal_patterns='Monthly', inverse=False, top_k=3, num_kernels=6, enc_in=7, dec_in=7, c_out=7, d_model=64, n_heads=8, e_layers=3, d_layers=1, d_ff=64, moving_avg=25, factor=1, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, use_gpu=True, gpu=1, loss='cross_entropy', optimizer='adam', lr=0.001, weight_decay=0.0, batch_size=8, epoch=15, cuda='cuda:1', save_dir='/SSD/lz/time_series_label_noise/result', save_csv_name='timesnet_ucr_supervised_0801_', classifier='linear', classifier_input=128)

## Data Preprocessing

In [4]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

train = train.drop('subject', axis=1)
test = test.drop('subject', axis=1)

labelNames = list(np.unique(train['Activity']))
labelNames.sort()
train['Activity'] = train['Activity'].apply(lambda x: labelNames.index(x))
test['Activity'] = test['Activity'].apply(lambda x: labelNames.index(x))

In [5]:
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,2
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,2
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,2
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,2
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,2


In [6]:
test.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,2
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,2


In [7]:
x_train = train.iloc[:, :-1].to_numpy(dtype=np.float32)
y_train = train.iloc[:, -1].to_numpy(dtype=np.float32)

x_test = test.iloc[:, :-1].to_numpy(dtype=np.float32)
y_test = test.iloc[:, -1].to_numpy(dtype=np.float32)

num_classes = len(np.unique(y_train))

In [8]:
x_train = x_train[:, :, np.newaxis]
x_test = x_test[:, :, np.newaxis]

In [9]:
args.num_classes = num_classes
args.seq_len = x_train.shape[1]
args.input_size = x_train.shape[2]

args.enc_in = x_train.shape[2]

while x_train.shape[0] * 0.6 < args.batch_size:
    args.batch_size = args.batch_size // 2

print("args.batch_size = ", args.batch_size, ", x_train.shape = ", x_train.shape, ", x_test.shape = ", x_test.shape)

args.batch_size =  8 , x_train.shape =  (7352, 561, 1) , x_test.shape =  (2947, 561, 1)


## Modelling

In [10]:
import torch
import torch.nn as nn


class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res

In [11]:
import torch
import torch.nn as nn
import math


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=25000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft


def FFT_for_Period(x, k=2):
    # [B, T, C]
    xf = torch.fft.rfft(x, dim=1)
    # find period by amplitudes
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list

    # print("period.shape = ", period.shape, top_list.shape, top_list, period)
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.k = configs.top_k
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs.d_model, configs.d_ff,
                               num_kernels=configs.num_kernels),
            nn.GELU(),
            Inception_Block_V1(configs.d_ff, configs.d_model,
                               num_kernels=configs.num_kernels)
        )

    def forward(self, x):
        # print("Input shape:", x.shape)
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        # print("period_list shape = ", period_list.shape, period_list)

        # print("period_list period_weight shape:", period_list.shape, period_weight.shape, self.k, self.seq_len, self.pred_len)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (
                                 ((self.seq_len + self.pred_len) // period) + 1) * period

                # print("length = ", length, self.seq_len, self.pred_len, period)
                padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)

                # print("padding x shape = ", padding.shape, x.shape)
                out = torch.cat([x, padding], dim=1)
                # print("padding out shape = ", out.shape)
            else:
                length = (self.seq_len + self.pred_len)
                out = x

            # print("out.shape = ", out.shape, length, period, length // period, N )
            # reshape
            out = out.reshape(B, length // period, period,
                              N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res


class TimesNetModel(nn.Module):
    """
    Paper link: https://openreview.net/pdf?id=ju_Uqw384Oq
    """

    def __init__(self, configs):
        super(TimesNetModel, self).__init__()
        self.configs = configs
        self.model = nn.ModuleList([TimesBlock(configs)
                                    for _ in range(configs.e_layers)])
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                           configs.dropout)
        self.layer = configs.e_layers
        self.layer_norm = nn.LayerNorm(configs.d_model)
        self.act = F.gelu
        self.dropout = nn.Dropout(configs.dropout)
        self.projection = nn.Linear(
            configs.d_model * configs.seq_len, configs.num_classes)

    def classification(self, x_enc, x_mark_enc):
        # embedding
        enc_out = self.enc_embedding(x_enc, None)  # [B,T,C]
        # TimesNet
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))

        # Output
        # the output transformer encoder/decoder embeddings don't include non-linearity
        output = self.act(enc_out)
        output = self.dropout(output)
        # zero-out padding embeddings
        output = output * x_mark_enc.unsqueeze(-1)
        # (batch_size, seq_length * d_model)
        output = output.reshape(output.shape[0], -1)
        output = self.projection(output)  # (batch_size, num_classes)
        return output

    def forward(self, x_enc, x_mark_enc, x_dec=None, x_mark_dec=None, mask=None):
        dec_out = self.classification(x_enc, x_mark_enc)
        return dec_out  # [B, N]


## Training

In [13]:
model = TimesNetModel(configs=args)
model

TimesNetModel(
  (model): ModuleList(
    (0-2): 3 x TimesBlock(
      (conv): Sequential(
        (0): Inception_Block_V1(
          (kernels): ModuleList(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
            (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
            (4): Conv2d(64, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
            (5): Conv2d(64, 64, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
          )
        )
        (1): GELU(approximate='none')
        (2): Inception_Block_V1(
          (kernels): ModuleList(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))


In [14]:
device = 'cuda'
model = model.to(device)

In [15]:
if args.normalize_way == 'single':
    x_train = normalize(x_train)
    x_test = normalize(x_test)

train_set = HARDataset(torch.from_numpy(x_train).type(torch.FloatTensor).to(device).permute(0,2,1),
                        torch.from_numpy(y_train).type(torch.FloatTensor).to(device).to(torch.int64))
test_set = HARDataset(torch.from_numpy(x_test).type(torch.FloatTensor).to(device).permute(0,2,1),
                        torch.from_numpy(y_test).type(torch.FloatTensor).to(device).to(torch.int64))

train_loader = DataLoader(train_set, batch_size=args.batch_size, num_workers=0, drop_last=True, collate_fn=lambda x: collate_fn(x, device, max_len=args.seq_len))
test_loader = DataLoader(test_set, batch_size=args.batch_size, num_workers=0, collate_fn=lambda x: collate_fn(x, device, max_len=args.seq_len))

In [16]:
loss = nn.CrossEntropyLoss().to(device)

model_init_state = model.state_dict()

if args.optimizer == 'adam':
    optimizer = torch.optim.Adam([{'params': model.parameters()}],
                                    lr=args.lr, weight_decay=args.weight_decay)
elif args.optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

losses = []
test_accuracies = []
train_time = 0.0
end_val_epochs = []

In [17]:
import torch
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

model.load_state_dict(model_init_state)
# Define optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

train_loss = []
train_accuracy = []
num_steps = args.epoch // args.batch_size

last_loss = float('inf')
stop_count = 0
increase_count = 0

num_steps = train_set.__len__() // args.batch_size

min_val_loss = float('inf')
test_accuracy = 0
end_val_epoch = 0

for epoch in range(args.epoch):
    print('Epoch {}/{}'.format(epoch + 1, args.epoch))

    epoch_train_loss = 0
    epoch_train_acc = 0
    num_iterations = 0

    model.train()

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{args.epoch}", unit="batch") as pbar:
        for x, y, padding_x_mask in train_loader:
            optimizer.zero_grad()

            # Forward pass
            pred = model(x, padding_x_mask)
            step_loss = loss(pred, y)

            # Backward pass and optimize
            step_loss.backward()
            optimizer.step()

            # Accumulate training loss and accuracy
            epoch_train_loss += step_loss.item()
            epoch_train_acc += torch.sum(torch.argmax(pred.data, axis=1) == y) / len(y)

            num_iterations += 1
            pbar.update(1)  # Update progress bar for each batch

    # Average train loss and accuracy over all steps
    epoch_train_loss /= num_steps
    epoch_train_acc /= num_steps

    # Adjust learning rate based on training loss
    scheduler.step(epoch_train_loss)

    print("Train loss: {} , Train accuracy : {}".format(epoch_train_loss, epoch_train_acc))
    print("Learning rate after epoch {}: {}".format(epoch, optimizer.param_groups[0]['lr']))


f:\Program Files\Anaconda\envs\py310\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/15


Epoch 1/15: 100%|██████████| 919/919 [01:57<00:00,  7.80batch/s]


Train loss: 27.233834809398747 , Train accuracy : 0.1328890025615692
Learning rate after epoch 0: 0.01
Epoch 2/15


Epoch 2/15: 100%|██████████| 919/919 [01:54<00:00,  8.05batch/s]


...

Train loss: 3.8983251967419736 , Train accuracy : 0.33093035221099854
Learning rate after epoch 3: 0.01
Epoch 5/15


Epoch 5/15: 100%|██████████| 919/919 [01:58<00:00,  7.76batch/s]


Train loss: 24.822907256924655 , Train accuracy : 0.4443688690662384
Learning rate after epoch 4: 0.01
Epoch 6/15


Epoch 6/15:  33%|███▎      | 305/919 [00:39<01:19,  7.76batch/s]


KeyboardInterrupt: 

## Testing

In [ ]:
import torch
import torchmetrics
from torchmetrics import Accuracy, F1Score, Precision, Recall, ConfusionMatrix, AUROC

# Initialize metrics
accuracy_metric = Accuracy().to(device)
f1_score_metric = F1Score(average='macro', num_classes=args.num_classes).to(device)
precision_metric = Precision(average='macro', num_classes=args.num_classes).to(device)
recall_metric = Recall(average='macro', num_classes=args.num_classes).to(device)
confusion_matrix_metric = ConfusionMatrix(num_classes=args.num_classes).to(device)
auroc_metric = AUROC(num_classes=args.num_classes).to(device)


model.eval()  # Set the model to evaluation mode
test_loss = 0

with torch.no_grad():  # No need to compute gradients for evaluation
    for x, y, padding_x_mask in test_loader:
        # Move inputs and targets to the correct device
        x, y = x.to(device), y.to(device)

        # Forward pass: compute predicted outputs by passing inputs to the model
        pred = model(x, padding_x_mask)

        # Calculate the loss
        loss = loss(pred, y)
        test_loss += loss.item()  # Accumulate loss

        # Update metrics
        accuracy_metric.update(pred, y)
        f1_score_metric.update(pred, y)
        precision_metric.update(pred, y)
        recall_metric.update(pred, y)
        confusion_matrix_metric.update(pred, y)
        auroc_metric.update(pred, y)

# Compute average test loss and finalize metrics
test_loss /= len(test_loader)
test_accuracy = accuracy_metric.compute().item()
test_f1_score = f1_score_metric.compute().item()
test_precision = precision_metric.compute().item()
test_recall = recall_metric.compute().item()
test_confusion_matrix = confusion_matrix_metric.compute().cpu().numpy()
test_auroc = auroc_metric.compute().item()

# Reset metrics for future evaluations
accuracy_metric.reset()
f1_score_metric.reset()
precision_metric.reset()
recall_metric.reset()
confusion_matrix_metric.reset()
auroc_metric.reset()

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1_score:.4f} Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test Confusion Matrix: {test_confusion_matrix}, Test AUROC: {test_auroc:.4f}")


Test Loss: 0.7468, Test Accuracy: 0.6763
Final Test Loss: 0.7468, Final Test Accuracy: 0.6763
